# Optimiser un code python

La plus part du temps vous n'aurez jamais besoin d'optimiser un code python, tellement les problemes viendront de la base de donnée, des API, des librairies etc... 

Et même quand vous optimiserez du code, il s'agira plus d'astuce fonctionnel, de mise en cache bien placée, d'utilisation de l'outil adapté, de solution de parallélisation simple que de connaissance profonde du langage. 

D'ailleurs pourquoi Python est aussi lent ? Souvent on dit que Python est lent et que si tu veux utiliesr un truc rapide code directement en C etc... mais en vrai pourquoi Python est plus lent qu'un autre ? 

On verra ça plus tard.

Surtout avant d'optimiser un code il faut le faire fonctionner correctement "premature optimization is the root of all evil". 

La méthode qui marche pour écrire un code performant en Python est déjà dans la [doc officielle] (https://wiki.python.org/moin/PythonSpeed/PerformanceTips) : 

1. Get it right.
2. Test it's right.
3. Profile if slow.
4. Optimise.
5. Repeat from 2.

### Les profiler 

Les profiler, ceux inclus de base : [profile, et cProfile](https://docs.python.org/2/library/profile.html) dans le TP on va utiliser [pyinstrument](https://github.com/joerick/pyinstrument). 

Il y a aussi [line_profiler](https://github.com/rkern/line_profiler) et [pyflame](https://github.com/uber-archive/pyflame) mais ils ne sont plus maintenu. 

Profiler c'est une étape éssentiel de l'optimisation car sinon on risque fort d'optimiser un truc inutile. 

Une fois que vous aurez profilez vous verrez bien que le probleme vient de la DB, probablement des JOIN stupides, mais mettons que pour une fois ce ne soit pas le cas, alors, il faut alors optimiser le code ! 


### Les optimisations qu'on a vu jusqu'à maintenant

#### Les slots
Python permet de définir des slots, lorsqu'un des slots sont définis alors Python va allouer une quantité statique de mémoire pour stocker  des attributs au lieu de stocker les attributs dans un dict, l'accés au atributs sera plus rapide et les attributs seront stockées dans un array au lieu d'être stocké dans un dict ce qui est plus économe en mémoire, moins de mémoire consommé dans le programme c'est un GC qui va se lancer moins fréquemment et donc de la perf de gagnée. 


```python
class MyClass(object):
    __slots__ = ['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()
```
#### Les générateurs
Les générateurs aussi apportent un gain de performance en permettant déconomiser la mémoire. Dans l'exemple plus bas les listes n'existent jamais toutes les 3 en même temps en mémoire mais son créée, renvoyée, traitée, colléctée (probablement) l'une aprés l'autre. 

```python
def huge_lists():
    yield [0] * 800000000
    yield [1] * 800000000
    yield [2] * 800000000

total_len = 0
for l in huge_lists():
    total_len += len(l)
```
#### Le cache
Une technique d'optimisation bien commune est l'utilisation de cache pour éviter de dépenser des ressources pour obtenir un résultat qu'on a déjà obtenu précédemment. La module [functools](https://docs.python.org/fr/3.8/library/functools.html) de la librairie standard fournit `lru_cache` et `cached_property`. [Django](https://docs.djangoproject.com/fr/2.2/topics/cache/) fournit une fonctionnalité de cache, il existe une exension [Flask](https://flask-caching.readthedocs.io/en/latest/) qui permet entre autre de s'appuyer sur Redis pour le cache, il existe aussi plusieurs client en Python pour [memcached](https://memcached.org/). 


### Les optimisations à connaitre

##### Utiliser la bonne collection au bon moment

Dans les cas où le programme passe beaucoup de temps à checker si un élément existe dans une collection il vaut mieux utilier un set qu'une liste. En python les set sont implémenté comme des tables de hachage, quand un set enregiste un élément il calcule son hache, il s'en sert ensuite pour détérminer où l'élément sera stocké (par exemple si le hash de l'élément commence par un 1 alors il sera stocké dans un tableau à l'indice 1, peu importe), quand un set va vérifier qu'il contient un élément donné alors il va calculer son hache et s'en servir pour savoir où chercher; alors que pour faire la même chose une liste va parcourir tous ses éléments, et tester si ils sont égaux à l'élément donné. 

Si on a pas besoin de tester la présence d'élément dans une collection, alors les listes seront plus éfficaces, parce que les listes ne doivent pas maintenir de table de hachage.  

Si on veut rechercher des élément par rapport à un attribut en particulier, genre l'age, le nom ou l'id, le plus raisonnable c'est d'utiliser un dictionnaire, et l'attribut en particlier comme clé de ce dictionnaire. 

Si on fait rien de tout ça mais qu'on manipule des grandes collections de nombres alors le module array de la librairie standard, ou les array numpy, seront un ordre de grandeur plus efficaces que les listes natives python.

Dans SciPy existe aussi plusieurs collections adaptés à des cas particulier. 

- executer des taches parallelement 
- éviter les memory leaks
- utiliser d'autres implémentation que CPython ; cython, numba, etc... 
- quand faire du calcul parallelle devient dur : utiliser spark ou autre
- utiliser numpy si on manipule de grande listes
- utiliser Gunicorn ou autre avec Flask
- utiliser des jobs queue pour éxécuter des jobs de maniere asynchrone


### Les optimisations dont on s'en fout

[ici](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)
